In [50]:
import pandas as pd

instagram_crawling =pd.read_csv('C:/Users/junseok/Downloads/instagram_crawling.csv')


In [51]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np

In [52]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace(",", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

In [53]:
def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
    return nouns

def tokenize(df):
    tokenizer = Mecab(dicpath=r'C:/mecab/mecab-ko-dic')
    processed_data = []
    for sent in tqdm(df['text']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data
def save_processed_data(processed_data):
    with open("tokenized_data_", 'w', newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in processed_data:
            writer.writerow(data)
            


In [54]:
if __name__ == '__main__':
    df = pd.read_csv("C:/Users/junseok/Downloads/instagram_crawling.csv")
    df.columns=['id', 'text', 'hashtag']
    df.dropna(how='any')
    processed_data = tokenize(df)
    save_processed_data(processed_data)

100%|██████████| 3157/3157 [00:01<00:00, 2654.54it/s]


In [55]:
processed_data = [sent.strip().split(",") for sent in tqdm(open("tokenized_data_",'r',encoding='utf-8').readlines())]
processed_data = DataFrame(processed_data)
processed_data[0] = processed_data[0].replace("", np.nan)
processed_data = processed_data[processed_data[0].notnull()]
processed_data = processed_data.values.tolist()
processed_data2=[]
for i in processed_data:
    i = list(filter(None, i))
    processed_data2.append(i)
processed_data = processed_data2


100%|██████████| 3157/3157 [00:00<00:00, 242712.40it/s]


In [56]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [57]:
dictionary = corpora.Dictionary(processed_data)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in processed_data]

2022-03-07 11:04:53,393 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-07 11:04:53,492 : INFO : built Dictionary(12797 unique tokens: ['갈비', '거미', '고기', '디저트', '마늘']...) from 3056 documents (total 147465 corpus positions)
2022-03-07 11:04:53,493 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12797 unique tokens: ['갈비', '거미', '고기', '디저트', '마늘']...) from 3056 documents (total 147465 corpus positions)", 'datetime': '2022-03-07T11:04:53.493449', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2022-03-07 11:04:53,503 : INFO : discarding 5814 tokens: [('맛있', 1865), ('거장', 1), ('바글', 1), ('라치오', 1), ('무나물', 1), ('식객', 1), ('장정식', 1), ('허영', 1), ('미나리볶음', 1), ('구단', 1)]...
2022-03-07 11:04:53,504 : INFO : keeping 6983 tokens which were in no less than 2 and no more than 1528 (=50.0%) documents
2022-03-07 11:04:53,509 : INFO : resulting dictiona

In [58]:
num_topics = 20
chunksize = 3056
passes = 30
iterations = 500
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


2022-03-07 11:05:01,765 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-03-07 11:05:01,766 : INFO : using serial LDA version on this node
2022-03-07 11:05:01,775 : INFO : running online (multi-pass) LDA training, 20 topics, 30 passes over the supplied corpus of 3056 documents, updating model once every 3056 documents, evaluating perplexity every 3056 documents, iterating 500x with a convergence threshold of 0.001000
2022-03-07 11:05:04,564 : INFO : -11.185 per-word bound, 2329.0 perplexity estimate based on a held-out corpus of 3056 documents with 136864 words
2022-03-07 11:05:04,564 : INFO : PROGRESS: pass 0, at document #3056/3056
2022-03-07 11:05:07,103 : INFO : optimized alpha [0.04287464, 0.041004688, 0.04191534, 0.041429263, 0.042239543, 0.044792596, 0.043532927, 0.04140999, 0.04301528, 0.042697042, 0.04190354, 0.041937105, 0.041153986, 0.041766815, 0.041153

In [59]:
top_topics = model.top_topics(corpus ,50) #num_words = 50

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

#from pprint import pprint
#pprint(top_topics)

2022-03-07 11:06:05,401 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-03-07 11:06:05,412 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-03-07 11:06:05,423 : INFO : CorpusAccumulator accumulated stats from 3000 documents


Average topic coherence: -2.9435.


In [60]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [61]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'instagram_feed_review.html')